In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [159]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata

In [6]:
def get_player_stats(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    table_body = soup.select_one('#yw1 > table > tbody')
    rows = table_body.find_all('tr', class_=['odd', 'even'])
    
    players = []
    tbody = soup.select_one('#yw1 > table > tbody')
    rows = tbody.find_all('tr', class_=['odd', 'even'])
    
    for row in rows:
        cols = row.find_all('td')
        name_tag = cols[1].select_one('table > tbody > tr:nth-child(1) > td.hauptlink > div:nth-child(1) > span > a')
        position_tag = cols[1].select_one('table > tbody > tr:nth-child(1) > td.zentriert.rueckennummer.bg_Torwart')
        country_tag = cols[3].find('img')
        
        player = {
            'Player': name_tag['title'].strip() if name_tag else '',
            'Position': position_tag.text.strip() if position_tag else '',
            'Kit Number': cols[2].text.strip(),
            'Nationality': country_tag['alt'].strip() if country_tag else '',
            'In Squad': cols[4].text.strip(),
            'Appearances': cols[5].text.strip(),
            'Goals': cols[6].text.strip(),
            'Assists': cols[7].text.strip(),
            'Yellow Cards': cols[8].text.strip(),
            'Second Yellows': cols[9].text.strip(),
            'Straight Reds': cols[10].text.strip(),
            'Substituted On': cols[11].text.strip(),
            'Substituted Off': cols[12].text.strip(),
            'PPG': cols[13].text.strip(),
            'Minutes Played': cols[14].text.strip()
        }
        
        # Handle "Not used during this season" cases
        if 'Not used during this season' in cols[5].text.strip():
            player.update({
                'Appearances': '0',
                'Goals': '0',
                'Assists': '0',
                'Yellow Cards': '0',
                'Second Yellows': '0',
                'Straight Reds': '0',
                'Substituted On': '0',
                'Substituted Off': '0',
                'PPG': '0',
                'Minutes Played': '0'
            })
        
        players.append(player)
    
    return players

# Example URL for Real Madrid 2023 season
url = 'https://www.transfermarkt.com/real-madrid/leistungsdaten/verein/418/plus/1?reldata=%262023'
player_stats = get_player_stats(url)

# Convert to DataFrame
df = pd.DataFrame(player_stats)
df.head(30)

,Player,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,Second Yellows,Straight Reds,Substituted On,Substituted Off,PPG,Minutes Played
0,,,,,Goalkeeper,31,,8,5,-,-,-,-,-,-
1,,,,,Goalkeeper,24,,55,31,-,-,2,-,-,-
2,,,,,Goalkeeper,28,,50,20,-,-,2,-,-,1
3,,,,,Goalkeeper,21,,8,Not used during this season,-,-,-,-,-,-
4,,,,,Goalkeeper,21,,3,Not used during this season,-,-,-,-,-,-
5,,,,,Goalkeeper,19,,17,Not used during this season,-,-,-,-,-,-
6,,,,,Goalkeeper,18,,21,Not used during this season,-,-,-,-,-,-
7,,,,,Centre-Back,25,,15,13,-,-,-,-,-,7
8,,,,,Centre-Back,30,,50,48,2,3,8,-,-,4
9,,,,,Left-Back,28,,44,37,1,-,6,-,-,4


In [57]:
def get_player_stats_redo(base_url, team_name, team_id, season_year):
    headers = {'User-Agent': 'Mozilla/5.0'}
    url = f"{base_url}/{team_name}/leistungsdaten/verein/{team_id}/plus/1?reldata=%26{season_year}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    match = re.search(r'reldata=%26(\d{4})', url)
    if match:
        year = int(match.group(1))
        season = f"{year}/{year + 1}"
    else:
        season = 'Unknown'

    # Extract the club name from the page header or URL (assuming it's in the URL for simplicity)
    club_name = soup.find('h1').text.strip() if soup.find('h1') else 'Unknown'

    
    players = []
    tbody = soup.select_one('#yw1 > table > tbody')
    rows = tbody.find_all('tr', class_=['odd', 'even'])
    
    for row in rows:
        cols = row.find_all('td')
        
        # Extracting the player name from the specific span class and a tag
        name_tag = row.select_one('span.hide-for-small > a')
        player_name = name_tag.text.strip() if name_tag else ''
            
        # Extracting the position
        position = cols[4].text.strip() if len(cols) > 4 else ''
        
        # Extracting the kit number
        kit_number = cols[0].text.strip() if len(cols) > 0 else ''
        
        # Extracting the nationality using the correct selector
        nationality_tag = row.select_one('td:nth-child(4) > img')
        nationality = nationality_tag['title'].strip() if nationality_tag else ''

        age = cols[5].text.strip() if len(cols) > 5 else ''
        
        # Extracting other statistics
        in_squad = cols[7].text.strip() if len(cols) > 7 else ''
        appearances = cols[8].text.strip() if len(cols) > 8 else ''
        goals = cols[9].text.strip() if len(cols) > 9 else ''
        assists = cols[10].text.strip() if len(cols) > 10 else ''
        yellow_cards = cols[11].text.strip() if len(cols) > 11 else ''
        second_yellows = cols[12].text.strip() if len(cols) > 12 else ''
        straight_reds = cols[13].text.strip() if len(cols) > 13 else ''
        substituted_on = cols[14].text.strip() if len(cols) > 14 else ''
        substituted_off = cols[15].text.strip() if len(cols) > 14 else ''
        ppg = cols[16].text.strip() if len(cols) > 14 else ''
        minutes_played = cols[17].text.strip() if len(cols) > 14 else ''
        
        # Handle "Not used during this season" cases
        if 'Not used during this season' in appearances:
            appearances = '0'
            goals = '0'
            assists = '0'
            yellow_cards = '0'
            second_yellows = '0'
            straight_reds = '0'
            substituted_on = '0'
            substituted_off = '0'
            ppg = '0'
            minutes_played = '0'
        
        player = {
            'Player': player_name,
            'Age': age,
            'Position': position,
            'Kit Number': kit_number,
            'Nationality': nationality,
            'In Squad': in_squad,
            'Appearances': appearances,
            'Goals': goals,
            'Assists': assists,
            'Yellow Cards': yellow_cards,
            'Second Yellows': second_yellows,
            'Straight Reds': straight_reds,
            'Substituted On': substituted_on,
            'Substituted Off': substituted_off,
            'PPG': ppg,
            'Minutes Played': minutes_played,
            'Club': club_name,
            'Season': season
        }
        
        players.append(player)
    
    return players

# Example usage for Real Madrid 2023 season
base_url = 'https://www.transfermarkt.com'
team_name = 'real-madrid'
team_id = 418
season_year = 2022
player_stats = get_player_stats_redo(base_url, team_name, team_id, season_year)

# Convert to DataFrame
df = pd.DataFrame(player_stats)
df.head(100)

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,Second Yellows,Straight Reds,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season
0,Thibaut Courtois,30,Goalkeeper,1,Belgium,50,49,-,-,1,-,-,-,-,2.14,4.470',Real Madrid,2022/2023
1,Andriy Lunin,23,Goalkeeper,13,Ukraine,59,12,-,-,-,-,-,-,-,2.17,1.080',Real Madrid,2022/2023
2,Lucas Cañizares,20,Goalkeeper,30,Spain,11,0,0,0,0,0,0,0,0,0,0,Real Madrid,2022/2023
3,Mario de Luis,20,Goalkeeper,43,Spain,1,0,0,0,0,0,0,0,0,0,0,Real Madrid,2022/2023
4,Luis López,21,Goalkeeper,26,Spain,52,0,0,0,0,0,0,0,0,0,0,Real Madrid,2022/2023
5,Diego Piñeiro,18,Goalkeeper,38,Spain,1,0,0,0,0,0,0,0,0,0,0,Real Madrid,2022/2023
6,Fran González,17,Goalkeeper,44,Spain,1,0,0,0,0,0,0,0,0,0,0,Real Madrid,2022/2023
7,Éder Militão,24,Centre-Back,3,Brazil,55,51,7,1,9,-,-,3,6,2.12,4.231',Real Madrid,2022/2023
8,Antonio Rüdiger,29,Centre-Back,22,Germany,58,53,2,-,2,-,-,14,2,2.08,3.848',Real Madrid,2022/2023
9,Ferland Mendy,27,Left-Back,23,France,36,28,-,1,4,-,-,3,13,2.25,2.064',Real Madrid,2022/2023


In [58]:
def collect_data_for_multiple_seasons(base_url, team_name, team_id, season_years):
    all_players = []
    
    for year in season_years:
        player_stats = get_player_stats_redo(base_url, team_name, team_id, year)
        all_players.extend(player_stats)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_players)
    
    # Save to CSV
    df.to_csv(f'{team_name}_multiple_seasons_stats.csv', index=False)
    return df

In [59]:
# Example usage for Real Madrid for specific seasons
base_url = 'https://www.transfermarkt.com'
team_name = 'fc-barcelona'
team_id = 131
season_years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]  # List of specific seasons you want to collect data for

df = collect_data_for_multiple_seasons(base_url, team_name, team_id, season_years)
df.head(100)

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,Second Yellows,Straight Reds,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season
0,Fernando Pacheco,18,Goalkeeper,32,Spain,1,0,0,0,0,0,0,0,0,0,0,Real Madrid,2010/2011
1,Antonio Adán,23,Goalkeeper,13,Spain,34,5,-,-,-,-,-,2,1,2.40,395',Real Madrid,2010/2011
2,Jesús Fernández,22,Goalkeeper,28,Spain,1,1,-,-,-,-,-,1,-,3.00,13',Real Madrid,2010/2011
3,Tomás Mejías,21,Goalkeeper,40,Spain,1,1,-,-,-,-,-,1,-,3.00,6',Real Madrid,2010/2011
4,Iker Casillas,29,Goalkeeper,1,Spain,54,54,-,-,4,-,1,-,-,2.39,4.802',Real Madrid,2010/2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Xabi Alonso,30,Defensive Midfield,14,Spain,49,47,-,8,18,-,-,6,8,1.94,3.793',Real Madrid,2012/2013
96,Michael Essien,29,Defensive Midfield,15,Ghana,43,35,2,1,7,-,-,4,7,2.23,2.702',Real Madrid,2012/2013
97,Lassana Diarra,27,Defensive Midfield,24,France,4,2,-,-,-,-,-,-,2,0.50,110',Real Madrid,2012/2013
98,Álvaro Morata,19,Centre-Forward,29,Spain,28,15,2,3,3,-,-,8,4,2.53,579',Real Madrid,2012/2013


In [60]:
df.head(50)

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,Second Yellows,Straight Reds,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season
0,Fernando Pacheco,18,Goalkeeper,32,Spain,1,0,0,0,0,0,0,0,0,0,0,Real Madrid,2010/2011
1,Antonio Adán,23,Goalkeeper,13,Spain,34,5,-,-,-,-,-,2,1,2.40,395',Real Madrid,2010/2011
2,Jesús Fernández,22,Goalkeeper,28,Spain,1,1,-,-,-,-,-,1,-,3.00,13',Real Madrid,2010/2011
3,Tomás Mejías,21,Goalkeeper,40,Spain,1,1,-,-,-,-,-,1,-,3.00,6',Real Madrid,2010/2011
4,Iker Casillas,29,Goalkeeper,1,Spain,54,54,-,-,4,-,1,-,-,2.39,4.802',Real Madrid,2010/2011
5,Jerzy Dudek,37,Goalkeeper,25,Poland,27,2,-,-,-,-,-,-,2,3.00,122',Real Madrid,2010/2011
6,Nacho Fernández,20,Centre-Back,35,Spain,3,2,-,-,-,-,-,-,1,1.50,142',Real Madrid,2010/2011
7,Sergio Ramos,24,Centre-Back,4,Spain,47,46,4,3,17,1,1,1,1,2.33,4.050',Real Madrid,2010/2011
8,Raúl Albiol,24,Centre-Back,18,Spain,46,32,-,-,7,-,1,8,3,2.41,2.151',Real Madrid,2010/2011
9,Marcelo,22,Left-Back,12,Brazil,53,50,5,10,8,-,-,3,5,2.50,4.239',Real Madrid,2010/2011


In [61]:
#need to get all player id's but might have to make multidimensional array for each season due to relegations and what not and the array being completely different for each league

In [62]:
###Next section for getting the teams, ids, placement, and season###

In [74]:
# Define the URL
url = "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id=2023"

# Extract the year from the URL using regex
match = re.search(r'saison_id=(\d{4})', url)
if match:
    year = int(match.group(1))
    season = f"{year}/{year + 1}"
else:
    raise ValueError("Year not found in the URL")

# Set up headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Fetch the HTML content with headers
response = requests.get(url, headers=headers)
response.raise_for_status()  # Ensure we notice bad responses

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the team data
table = soup.select_one('#yw1 > table > tbody')

# Extract the required data
team_data = []

if table:
    rows = table.find_all('tr')
    for row in rows:
        try:
            # Team name and ID
            team_anchor = row.select_one('td.no-border-links.hauptlink > a')
            team_name = team_anchor['title']
            
            # Extract team ID from the href attribute using regex
            team_href = team_anchor['href']
            team_id_match = re.search(r'/verein/(\d+)/', team_href)
            team_id = team_id_match.group(1) if team_id_match else 'N/A'

            # Extract the team link name
            team_link_name = team_href.split('/')[1]

            # Team placement
            team_placement = row.select_one('td.rechts.hauptlink').text.strip()

            team_data.append({
                'Team Name': team_name,
                'Team ID': team_id,
                'Placement': team_placement,
                'Season': season,
                'Team Link Name': team_link_name
            })
        except AttributeError as e:
            print(f"Error extracting data for a row: {e}")

# Convert the data to a DataFrame
placement_id_df = pd.DataFrame(team_data)

placement_id_df.head(20)

,Team Name,Team ID,Placement,Season,Team Link Name
0,Real Madrid,418,1,2023/2024,real-madrid
1,FC Barcelona,131,2,2023/2024,fc-barcelona
2,Girona FC,12321,3,2023/2024,fc-girona
3,Atlético de Madrid,13,4,2023/2024,atletico-madrid
4,Athletic Bilbao,621,5,2023/2024,athletic-bilbao
5,Real Sociedad,681,6,2023/2024,real-sociedad-san-sebastian
6,Real Betis Balompié,150,7,2023/2024,real-betis-sevilla
7,Villarreal CF,1050,8,2023/2024,fc-villarreal
8,Valencia CF,1049,9,2023/2024,fc-valencia
9,Deportivo Alavés,1108,10,2023/2024,deportivo-alaves


In [78]:
##Same as previous function but loop for all the season I need
# Define the base URL and headers
base_url = "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id={year}"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Initialize an empty DataFrame to store all the data
laliga_standings = pd.DataFrame(columns=['Team Name', 'Team ID', 'Placement', 'Season', 'Team Link Name'])

# Loop over each year from 2023 to 2008
for year in range(2023, 2007, -1):
    url = base_url.format(year=year)
    
    # Extract the season string
    season = f"{year}/{year + 1}"
    
    # Fetch the HTML content with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Ensure we notice bad responses
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table containing the team data
    table = soup.select_one('#yw1 > table > tbody')
    
    # Extract the required data
    team_data = []
    
    if table:
        rows = table.find_all('tr')
        for row in rows:
            try:
                # Team name and ID
                team_anchor = row.select_one('td.no-border-links.hauptlink > a')
                team_name = team_anchor['title']
                
                # Extract team ID from the href attribute using regex
                team_href = team_anchor['href']
                team_id_match = re.search(r'/verein/(\d+)/', team_href)
                team_id = team_id_match.group(1) if team_id_match else 'N/A'
    
                # Extract the team link name
                team_link_name = team_href.split('/')[1]
    
                # Team placement
                team_placement = row.select_one('td.rechts.hauptlink').text.strip()
    
                team_data.append({
                    'Team Name': team_name,
                    'Team ID': team_id,
                    'Placement': team_placement,
                    'Season': season,
                    'Team Link Name': team_link_name
                })
            except AttributeError as e:
                print(f"Error extracting data for a row: {e}")
    
    # Convert the data to a DataFrame and append to the main DataFrame
    year_df = pd.DataFrame(team_data)
    laliga_standings = pd.concat([laliga_standings, year_df], ignore_index=True)



In [77]:
laliga_standings.tail(60)

,Team Name,Team ID,Placement,Season,Team Link Name
260,FC Barcelona,131,1,2010/2011,fc-barcelona
261,Real Madrid,418,2,2010/2011,real-madrid
262,Valencia CF,1049,3,2010/2011,fc-valencia
263,Villarreal CF,1050,4,2010/2011,fc-villarreal
264,Sevilla FC,368,5,2010/2011,fc-sevilla
265,Athletic Bilbao,621,6,2010/2011,athletic-bilbao
266,Atlético de Madrid,13,7,2010/2011,atletico-madrid
267,RCD Espanyol Barcelona,714,8,2010/2011,espanyol-barcelona
268,CA Osasuna,331,9,2010/2011,ca-osasuna
269,Sporting Gijón,2448,10,2010/2011,sporting-gijon


In [83]:
# Define the base URL and headers
base_url = "https://www.transfermarkt.com/{league}/tabelle/wettbewerb/{competition}?saison_id={year}"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# List of leagues and their respective competition codes
leagues = {
    'laliga': 'ES1',
    'premier-league': 'GB1',
    'bundesliga': 'L1',
    'serie-a': 'IT1',
    'ligue-1': 'FR1'
}

# Initialize a dictionary to store DataFrames for each league
league_dfs = {league: pd.DataFrame(columns=[
    'Team Name', 'Team ID', 'Placement', 'Season', 'Team Link Name', 'League', 'Champions League', 'Domestic Cup']) 
    for league in leagues}

# Loop over each league
for league, competition in leagues.items():
    # Loop over each year from 2023 to 2008
    for year in range(2023, 2007, -1):
        url = base_url.format(league=league, competition=competition, year=year)
        
        # Extract the season string
        season = f"{year}/{year + 1}"
        
        # Fetch the HTML content with headers
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Ensure we notice bad responses
        
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table containing the team data
        table = soup.select_one('#yw1 > table > tbody')
        
        # Extract the required data
        team_data = []
        
        if table:
            rows = table.find_all('tr')
            for row in rows:
                try:
                    # Team name and ID
                    team_anchor = row.select_one('td.no-border-links.hauptlink > a')
                    team_name = team_anchor['title']
                    
                    # Extract team ID from the href attribute using regex
                    team_href = team_anchor['href']
                    team_id_match = re.search(r'/verein/(\d+)/', team_href)
                    team_id = team_id_match.group(1) if team_id_match else 'N/A'
        
                    # Extract the team link name
                    team_link_name = team_href.split('/')[1]
        
                    # Team placement
                    team_placement = row.select_one('td.rechts.hauptlink').text.strip()
        
                    team_data.append({
                        'Team Name': team_name,
                        'Team ID': team_id,
                        'Placement': team_placement,
                        'Season': season,
                        'Team Link Name': team_link_name,
                        'League': league,
                        'Champions League': 0,  # Default to 0
                        'Domestic Cup': 0       # Default to 0
                    })
                except AttributeError as e:
                    print(f"Error extracting data for a row: {e}")
        
        # Convert the data to a DataFrame and append to the league's DataFrame
        year_df = pd.DataFrame(team_data)
        league_dfs[league] = pd.concat([league_dfs[league], year_df], ignore_index=True)

In [84]:
league_dfs['ligue'].tail(60)

,Team Name,Team ID,Placement,Season,Team Link Name,League,Champions League,Domestic Cup
260,Manchester United,985,1,2010/2011,manchester-united,premier-league,0,0
261,Chelsea FC,631,2,2010/2011,fc-chelsea,premier-league,0,0
262,Manchester City,281,3,2010/2011,manchester-city,premier-league,0,0
263,Arsenal FC,11,4,2010/2011,fc-arsenal,premier-league,0,0
264,Tottenham Hotspur,148,5,2010/2011,tottenham-hotspur,premier-league,0,0
265,Liverpool FC,31,6,2010/2011,fc-liverpool,premier-league,0,0
266,Everton FC,29,7,2010/2011,fc-everton,premier-league,0,0
267,Fulham FC,931,8,2010/2011,fc-fulham,premier-league,0,0
268,Aston Villa,405,9,2010/2011,aston-villa,premier-league,0,0
269,Sunderland AFC,289,10,2010/2011,afc-sunderland,premier-league,0,0


In [86]:
# Define the base URL for player data scraping
player_data_base_url = 'https://www.transfermarkt.com'

# Function to get player stats
def get_player_stats_redo(player_data_base_url, team_name, team_id, season_year, league, champions_league, domestic_cup):
    headers = {'User-Agent': 'Mozilla/5.0'}
    url = f"{player_data_base_url}/{team_name}/leistungsdaten/verein/{team_id}/plus/1?reldata=%26{season_year}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    match = re.search(r'reldata=%26(\d{4})', url)
    if match:
        year = int(match.group(1))
        season = f"{year}/{year + 1}"
    else:
        season = 'Unknown'

    club_name = soup.find('h1').text.strip() if soup.find('h1') else 'Unknown'
    
    players = []
    tbody = soup.select_one('#yw1 > table > tbody')
    
    if not tbody:
        print(f"No table body found for URL: {url}")
        return players
    
    rows = tbody.find_all('tr', class_=['odd', 'even'])
    
    for row in rows:
        cols = row.find_all('td')
        
        name_tag = row.select_one('span.hide-for-small > a')
        player_name = name_tag.text.strip() if name_tag else ''
            
        position = cols[4].text.strip() if len(cols) > 4 else ''
        kit_number = cols[0].text.strip() if len(cols) > 0 else ''
        nationality_tag = row.select_one('td:nth-child(4) > img')
        nationality = nationality_tag['title'].strip() if nationality_tag else ''

        age = cols[5].text.strip() if len(cols) > 5 else ''
        in_squad = cols[7].text.strip() if len(cols) > 7 else ''
        appearances = cols[8].text.strip() if len(cols) > 8 else ''
        goals = cols[9].text.strip() if len(cols) > 9 else ''
        assists = cols[10].text.strip() if len(cols) > 10 else ''
        yellow_cards = cols[11].text.strip() if len(cols) > 11 else ''
        second_yellows = cols[12].text.strip() if len(cols) > 12 else ''
        straight_reds = cols[13].text.strip() if len(cols) > 13 else ''
        substituted_on = cols[14].text.strip() if len(cols) > 14 else ''
        substituted_off = cols[15].text.strip() if len(cols) > 14 else ''
        ppg = cols[16].text.strip() if len(cols) > 14 else ''
        minutes_played = cols[17].text.strip() if len(cols) > 14 else ''
        
        if 'Not used during this season' in appearances:
            appearances = '0'
            goals = '0'
            assists = '0'
            yellow_cards = '0'
            second_yellows = '0'
            straight_reds = '0'
            substituted_on = '0'
            substituted_off = '0'
            ppg = '0'
            minutes_played = '0'
        
        player = {
            'Player': player_name,
            'Age': age,
            'Position': position,
            'Kit Number': kit_number,
            'Nationality': nationality,
            'In Squad': in_squad,
            'Appearances': appearances,
            'Goals': goals,
            'Assists': assists,
            'Yellow Cards': yellow_cards,
            'Second Yellows': second_yellows,
            'Straight Reds': straight_reds,
            'Substituted On': substituted_on,
            'Substituted Off': substituted_off,
            'PPG': ppg,
            'Minutes Played': minutes_played,
            'Club': club_name,
            'Season': season,
            'Team': team_name,
            'League': league,
            'Champions League': champions_league,
            'Domestic Cup': domestic_cup
        }
        
        players.append(player)
    
    return players

# Initialize a comprehensive DataFrame to store all player data
comprehensive_df = pd.DataFrame(columns=[
    'Player', 'Age', 'Position', 'Kit Number', 'Nationality', 'In Squad', 'Appearances', 'Goals', 
    'Assists', 'Yellow Cards', 'Second Yellows', 'Straight Reds', 'Substituted On', 'Substituted Off', 
    'PPG', 'Minutes Played', 'Club', 'Season', 'Team', 'League', 'Champions League', 'Domestic Cup'
])

# Loop over each league DataFrame to get player data
for league, df in league_dfs.items():
    grouped_df = df.groupby('Season')
    for season, group in grouped_df:
        for index, row in group.iterrows():
            team_name = row['Team Link Name']
            team_id = row['Team ID']
            season_year = int(season.split('/')[0])
            champions_league = row['Champions League']
            domestic_cup = row['Domestic Cup']
            player_stats = get_player_stats_redo(
                player_data_base_url, team_name, team_id, season_year, league, champions_league, domestic_cup
            )
            comprehensive_df = pd.concat([comprehensive_df, pd.DataFrame(player_stats)], ignore_index=True)

In [99]:
comprehensive_df.head(30)

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup
0,Oier Olazábal,18,Goalkeeper,-,Spain,2,1,-,-,-,...,-,-,"0,00",90',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
1,Víctor Valdés,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
2,José Manuel Pinto,32,Goalkeeper,13,Spain,51,11,-,-,1,...,-,-,2.18,990',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
3,Albert Jorquera,29,Goalkeeper,25,Spain,20,1,-,-,-,...,-,-,"0,00",90',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
4,Andreu Fontàs,18,Centre-Back,-,Spain,1,0,0,0,0,...,0,0,0,0,FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
5,Alberto Botía,19,Centre-Back,-,Spain,5,1,-,-,-,...,1,-,1.00,27',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
6,Martín Cáceres,21,Centre-Back,2,Uruguay,51,23,-,-,3,...,7,-,2.13,"1,523'",FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
7,Marc Muniesa,16,Centre-Back,-,Spain,2,1,-,-,-,...,1,-,"0,00",31',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
8,Gerard Piqué,21,Centre-Back,3,Spain,50,45,3,1,7,...,1,1,2.27,"3,932'",FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
9,Rafa Márquez,29,Centre-Back,4,Mexico,41,37,3,3,7,...,1,8,2.49,"3,020'",FC Barcelona,2008/2009,fc-barcelona,laliga,0,0


In [95]:
comprehensive_df.describe()

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup
count,58044,58044,58044,58044,58044,58044,58044,58044,58044,58044,...,58044,58044,58044,58044,58044,58044,58044,58044,58044,58044
unique,16147,34,16,100,166,67,66,61,34,24,...,39,41,282,4422,191,16,191,5,1,1
top,,19,Centre-Back,-,Spain,1,0,-,-,-,...,-,-,0,0,Udinese Calcio,2018/2019,udinese-calcio,serie-a,0,0
freq,1496,4266,10709,9719,7830,4120,7678,27144,26686,16638,...,12116,15092,11322,7678,724,3915,724,13681,58044,58044


In [97]:
###Some data cleaning
comprehensive_df['Minutes Played'] = comprehensive_df['Minutes Played'].str.replace('.', ',')

# Make a copy of the comprehensive DataFrame
comprehensive_df_copy = comprehensive_df.copy()

comprehensive_df_copy.head()

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup
0,Oier Olazábal,18,Goalkeeper,-,Spain,2,1,-,-,-,...,-,-,"0,00",90',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
1,Víctor Valdés,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
2,José Manuel Pinto,32,Goalkeeper,13,Spain,51,11,-,-,1,...,-,-,2.18,990',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
3,Albert Jorquera,29,Goalkeeper,25,Spain,20,1,-,-,-,...,-,-,"0,00",90',FC Barcelona,2008/2009,fc-barcelona,laliga,0,0
4,Andreu Fontàs,18,Centre-Back,-,Spain,1,0,0,0,0,...,0,0,0,0,FC Barcelona,2008/2009,fc-barcelona,laliga,0,0


In [139]:
###Some data cleaning. Remember I am editing the COPY

# Replace "Not in squad during this season" with 0 in the 'Appearances' column
comprehensive_df_copy['Appearances'] = comprehensive_df_copy['Appearances'].replace('Not in squad during this season', '0')

# Convert the 'Appearances' column to integers
comprehensive_df_copy['Appearances'] = comprehensive_df_copy['Appearances'].astype(int)

# Set the minimum number of appearances
min_appearances = 20

# Filter the DataFrame
filtered_df = comprehensive_df_copy[comprehensive_df_copy['Appearances'] >= min_appearances]

champions_league_winners = {
    '2023/2024': 'real-madrid',
    '2022/2023': 'manchester-city',
    '2021/2022': 'real-madrid',
    '2020/2021': 'fc-chelsea',
    '2019/2020': 'fc-bayern-munchen',
    '2018/2019': 'fc-liverpool',
    '2017/2018': 'real-madrid',
    '2016/2017': 'real-madrid',
    '2015/2016': 'real-madrid',
    '2014/2015': 'fc-barcelona',
    '2013/2014': 'real-madrid',
    '2012/2013': 'fc-bayern-munchen',
    '2011/2012': 'fc-chelsea',
    '2010/2011': 'fc-barcelona',
    '2009/2010': 'inter-mailand',
    '2008/2009': 'fc-barcelona'
}

# Function to update the Champions League column
def update_champions_league_winners(df, winners_dict):
    for season, winning_team in winners_dict.items():
        # Update the Champions League column
        df.loc[(df['Season'] == season) & (df['Team'] == winning_team), 'Champions League'] = 1
    return df


ucl_winners_df = update_champions_league_winners(filtered_df, champions_league_winners)


In [141]:
#incremental check

# Teams to check
teams_to_check = ['fc-chelsea', 'fc-bayern-munchen', 'manchester-city', 'fc-barcelona', 'borussia-dortmund']

# Function to check for teams in the DataFrame
def check_teams_in_df(df, teams):
    for team in teams:
        team_records = df[df['Team'] == team]
        print(f"Records for {team.capitalize()}:")
        print(team_records if not team_records.empty else f"No records found for {team.capitalize()}")
        print("\n" + "-"*50 + "\n")

# Check for teams in the original comprehensive DataFrame
check_teams_in_df(comprehensive_df, teams_to_check)

Records for Fc-chelsea:
                   Player Age        Position Kit Number     Nationality  \
10969      Carlo Cudicini  34      Goalkeeper          -           Italy   
10970           Petr Cech  26      Goalkeeper          1  Czech Republic   
10971         Rhys Taylor  18      Goalkeeper         30           Wales   
10972             Hilário  32      Goalkeeper         40        Portugal   
10973        Wayne Bridge  27       Left-Back          -         England   
...                   ...  ..             ...        ...             ...   
22669  David Datro Fofana  20  Centre-Forward          -   Cote d'Ivoire   
22670       Diego Moreira  18     Left Winger          -        Portugal   
22671       Mason Burstow  19  Centre-Forward          -         England   
22672      Ronnie Stutter  18  Centre-Forward          -         England   
22673      Tyrique George  17     Left Winger          -         England   

      In Squad                      Appearances Goals Assists Y

In [133]:
def search_team_season_records(df, team_name, season_year):
    """
    Search for player records of a specific team in a specific season.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to search within.
    team_name (str): The name of the team.
    season_year (str): The season year (e.g., '2022/2023').
    
    Returns:
    pd.DataFrame: Filtered DataFrame containing records for the specified team and season.
    """
    # Filter the DataFrame
    records = df[(df['Team'] == team_name) & (df['Season'] == season_year)]
    
    # Display the records
    print(records)
    
    return records

# Example usage
team_name = 'chelsea'
season_year = '2019/2020'
team_season_records = search_team_season_records(filtered_df, team_name, season_year)

Empty DataFrame
Columns: [Player, Age, Position, Kit Number, Nationality, In Squad, Appearances, Goals, Assists, Yellow Cards, Second Yellows, Straight Reds, Substituted On, Substituted Off, PPG, Minutes Played, Club, Season, Team, League, Champions League, Domestic Cup]
Index: []

[0 rows x 22 columns]


In [144]:
def search_champions_league_winners(df):
    """
    Search for player records where the 'Champions League' column is 1.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to search within.
    
    Returns:
    pd.DataFrame: Filtered DataFrame containing records where the 'Champions League' column is 1.
    """
    # Filter the DataFrame
    records = df[df['Champions League'] == 1]
    
    # Display the records
    print(records)
    
    return records

# Example usage
champions_league_winners_df = search_champions_league_winners(ucl_winners_df)

champions_league_winners_df.head(500)

                  Player Age            Position Kit Number      Nationality  \
1          Víctor Valdés  26          Goalkeeper          1            Spain   
6         Martín Cáceres  21         Centre-Back          2          Uruguay   
8           Gerard Piqué  21         Centre-Back          3            Spain   
9           Rafa Márquez  29         Centre-Back          4           Mexico   
10          Carles Puyol  30         Centre-Back          5            Spain   
...                  ...  ..                 ...        ...              ...   
33911  Esteban Cambiasso  28  Defensive Midfield         19        Argentina   
33919    Mario Balotelli  18      Centre-Forward         45            Italy   
33926       Samuel Eto'o  28      Centre-Forward          9         Cameroon   
33928       Diego Milito  30      Centre-Forward         22        Argentina   
33929       Goran Pandev  25      Second Striker         27  North Macedonia   

      In Squad  Appearances Goals Assis

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup
1,Víctor Valdés,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,0
6,Martín Cáceres,21,Centre-Back,2,Uruguay,51,23,-,-,3,...,7,-,2.13,"1,523'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,0
8,Gerard Piqué,21,Centre-Back,3,Spain,50,45,3,1,7,...,1,1,2.27,"3,932'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,0
9,Rafa Márquez,29,Centre-Back,4,Mexico,41,37,3,3,7,...,1,8,2.49,"3,020'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,0
10,Carles Puyol,30,Centre-Back,5,Spain,49,45,1,3,8,...,4,6,2.29,"3,768'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33911,Esteban Cambiasso,28,Defensive Midfield,19,Argentina,48,47,4,2,3,...,6,14,2.13,"3,590'",Inter Milan,2009/2010,inter-mailand,serie-a,1,0
33919,Mario Balotelli,18,Centre-Forward,45,Italy,44,40,11,9,10,...,21,13,2.25,"2,080'",Inter Milan,2009/2010,inter-mailand,serie-a,1,0
33926,Samuel Eto'o,28,Centre-Forward,9,Cameroon,48,48,16,9,8,...,5,10,2.06,"3,886'",Inter Milan,2009/2010,inter-mailand,serie-a,1,0
33928,Diego Milito,30,Centre-Forward,22,Argentina,53,52,30,8,3,...,5,18,2.17,"4,159'",Inter Milan,2009/2010,inter-mailand,serie-a,1,0


In [145]:
#domestic cup winners
fa_cup_winners = {
    '2023/2024': 'manchester-united',
    '2022/2023': 'manchester-city',
    '2021/2022': 'fc-liverpool',
    '2020/2021': 'leicester-city',
    '2019/2020': 'fc-arsenal',
    '2018/2019': 'manchester-city',
    '2017/2018': 'fc-chelsea',
    '2016/2017': 'fc-arsenal',
    '2015/2016': 'manchester-united',
    '2014/2015': 'fc-arsenal',
    '2013/2014': 'fc-arsenal',
    '2012/2013': 'wigan-athletic',
    '2011/2012': 'fc-chelsea',
    '2010/2011': 'manchester-city',
    '2009/2010': 'fc-chelsea',
    '2008/2009': 'fc-chelsea'
}

dfb_pokal_winners = {
    '2023/2024': 'bayer-04-leverkusen',
    '2022/2023': 'rasenballsport-leipzig',
    '2021/2022': 'rasenballsport-leipzig',
    '2020/2021': 'borussia-dortmund',
    '2019/2020': 'fc-bayern-munchen',
    '2018/2019': 'fc-bayern-munchen',
    '2017/2018': 'eintracht-frankfurt',
    '2016/2017': 'borussia-dortmund',
    '2015/2016': 'fc-bayern-munchen',
    '2014/2015': 'vfl-wolfsburg',
    '2013/2014': 'fc-bayern-munchen',
    '2012/2013': 'fc-bayern-munchen',
    '2011/2012': 'borussia-dortmund',
    '2010/2011': 'fc-schalke-04',
    '2009/2010': 'fc-bayern-munchen',
    '2008/2009': 'sv-werder-bremen'
}

copa_del_rey_winners = {
    '2023/2024': 'athletic-bilbao',
    '2022/2023': 'real-madrid',
    '2021/2022': 'real-betis-sevilla',
    '2020/2021': 'fc-barcelona',
    '2019/2020': 'real-sociedad-san-sebastian',
    '2018/2019': 'fc-valencia',
    '2017/2018': 'fc-barcelona',
    '2016/2017': 'fc-barcelona',
    '2015/2016': 'fc-barcelona',
    '2014/2015': 'fc-barcelona',
    '2013/2014': 'real-madrid',
    '2012/2013': 'atletico-madrid',
    '2011/2012': 'fc-barcelona',
    '2010/2011': 'real-madrid',
    '2009/2010': 'fc-sevilla',
    '2008/2009': 'fc-barcelona'
}

coupe_de_france_winners = {
    '2023/2024': 'fc-paris-saint-germain',
    '2022/2023': 'fc-toulouse',
    '2021/2022': 'fc-nantes',
    '2020/2021': 'fc-paris-saint-germain',
    '2019/2020': 'fc-paris-saint-germain',
    '2018/2019': 'fc-stade-rennes',
    '2017/2018': 'fc-paris-saint-germain',
    '2016/2017': 'fc-paris-saint-germain',
    '2015/2016': 'fc-paris-saint-germain',
    '2014/2015': 'fc-paris-saint-germain',
    '2013/2014': 'ea-guingamp',
    '2012/2013': 'fc-girondins-bordeaux',
    '2011/2012': 'olympique-lyon',
    '2010/2011': 'losc-lille',
    '2009/2010': 'fc-paris-saint-germain',
    '2008/2009': 'ea-guingamp'
}

coppa_italia_winners = {
    '2023/2024': 'juventus-turin',
    '2022/2023': 'inter-mailand',
    '2021/2022': 'inter-mailand',
    '2020/2021': 'juventus-turin',
    '2019/2020': 'ssc-neapel',
    '2018/2019': 'lazio-rom',
    '2017/2018': 'juventus-turin',
    '2016/2017': 'juventus-turin',
    '2015/2016': 'juventus-turin',
    '2014/2015': 'juventus-turin',
    '2013/2014': 'ssc-neapel',
    '2012/2013': 'lazio-rom',
    '2011/2012': 'ssc-neapel',
    '2010/2011': 'inter-mailand',
    '2009/2010': 'inter-mailand',
    '2008/2009': 'lazio-rom'
}

In [147]:
# Function to update the Domestic Cup column based on winners dictionaries
def update_domestic_cup_winners(df, winners_dict, column_name='Domestic Cup'):
    for season, winning_team in winners_dict.items():
        # Update the Domestic Cup column
        df.loc[(df['Season'] == season) & (df['Team'] == winning_team), column_name] = 1
    return df


# Assume ucl_winners_df is your DataFrame
# First, make sure the 'Domestic Cup' column exists
ucl_winners_df['Domestic Cup'] = 0

# Update the Domestic Cup column for each competition
ucl_winners_df = update_domestic_cup_winners(ucl_winners_df, coppa_italia_winners)
ucl_winners_df = update_domestic_cup_winners(ucl_winners_df, coupe_de_france_winners)
ucl_winners_df = update_domestic_cup_winners(ucl_winners_df, copa_del_rey_winners)
ucl_winners_df = update_domestic_cup_winners(ucl_winners_df, dfb_pokal_winners)
ucl_winners_df = update_domestic_cup_winners(ucl_winners_df, fa_cup_winners)

# Display the updated DataFrame
print(ucl_winners_df[ucl_winners_df['Domestic Cup'] == 1])

C:\Users\haiaj\AppData\Local\Temp\ipykernel_29624\1210425503.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucl_winners_df['Domestic Cup'] = 0


                  Player Age        Position Kit Number Nationality In Squad  \
1          Víctor Valdés  26      Goalkeeper          1       Spain       52   
6         Martín Cáceres  21     Centre-Back          2     Uruguay       51   
8           Gerard Piqué  21     Centre-Back          3       Spain       50   
9           Rafa Márquez  29     Centre-Back          4      Mexico       41   
10          Carles Puyol  30     Centre-Back          5       Spain       49   
...                  ...  ..             ...        ...         ...      ...   
57377    Ousmane Dembélé  26    Right Winger         10      France       45   
57378      Gonçalo Ramos  22  Centre-Forward          9    Portugal       50   
57379    Bradley Barcola  20     Left Winger         29      France       45   
57380  Randal Kolo Muani  24  Centre-Forward         23      France       46   
57381      Marco Asensio  27    Right Winger         11       Spain       37   

       Appearances Goals Assists Yellow

In [148]:
ucl_winners_df.head()

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted On,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup
1,Víctor Valdés,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1
6,Martín Cáceres,21,Centre-Back,2,Uruguay,51,23,-,-,3,...,7,-,2.13,"1,523'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1
8,Gerard Piqué,21,Centre-Back,3,Spain,50,45,3,1,7,...,1,1,2.27,"3,932'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1
9,Rafa Márquez,29,Centre-Back,4,Mexico,41,37,3,3,7,...,1,8,2.49,"3,020'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1
10,Carles Puyol,30,Centre-Back,5,Spain,49,45,1,3,8,...,4,6,2.29,"3,768'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1


In [221]:
# Add the 'TOTY' column with default value 0
ucl_winners_df['TOTY'] = 0

# '2023/2024': [
#         ('alisson', 'fc-liverpool'), 
#         ('jeremie frimpong', 'bayer-04-leverkusen'), 
#         ('ruben dias', 'manchester-city'), 
#         ('virgil van dijk', 'fc-liverpool'), 
#         ('theo hernandez', 'ac-mailand'), 
#         ('rodri', 'manchester-city'), 
#         ('jude bellingham', 'real-madrid'), 
#         ('kevin de bruyne', 'manchester-city'), 
#         ('lionel messi', 'inter-miami'), 
#         ('kylian mbappe', 'fc-paris-saint-germain'), 
#         ('erling haaland', 'manchester-city')
#     ],

toty_players = {
    '2022/2023': [
        ('thibaut courtois', 'real-madrid'), 
        ('achraf hakimi', 'fc-paris-saint-germain'), 
        ('virgil van dijk', 'fc-liverpool'), 
        ('eder militao', 'real-madrid'), 
        ('theo hernandez', 'ac-mailand'), 
        ('luka modric', 'real-madrid'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('jude bellingham', 'borussia-dortmund'), 
        ('kylian mbappe', 'fc-paris-saint-germain'), 
        ('lionel messi', 'fc-paris-saint-germain'), 
        ('karim benzema', 'real-madrid')
    ],
    '2021/2022': [
        ('gianluigi donnaruma', 'fc-paris-saint-germain'), 
        ('achraf hakimi', 'fc-paris-saint-germain'), 
        ('marquinhos', 'fc-paris-saint-germain'), 
        ('ruben dias', 'manchester-city'), 
        ('joao cancelo', 'manchester-city'), 
        ('jorginho', 'fc-chelsea'), 
        ('kante', 'fc-chelsea'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('kylian mbappe', 'fc-paris-saint-germain'), 
        ('robert lewandowski', 'fc-bayern-munchen'), 
        ('lionel messi', 'fc-paris-saint-germain')
    ],
    '2020/2021': [
        ('manuel neuer', 'fc-bayern-munchen'), 
        ('trent alexander-arnold', 'fc-liverpool'), 
        ('virgil van dijk', 'fc-liverpool'), 
        ('sergio ramos', 'real-madrid'), 
        ('alphonso davies', 'fc-bayern-munchen'), 
        ('joshua kimmich', 'fc-bayern-munchen'), 
        ('bruno fernandez', 'manchester-united'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('kylian mbappe', 'fc-paris-saint-germain'), 
        ('robert lewandowski', 'fc-bayern-munchen'), 
        ('cristiano ronaldo', 'juventus-turin')
    ],
    '2019/2020': [
        ('alisson', 'fc-liverpool'), 
        ('trent alexander-arnold', 'fc-liverpool'), 
        ('virgil van dijk', 'fc-liverpool'), 
        ('matthijs de ligt', 'juventus-turin'), 
        ('andrew robertson', 'fc-liverpool'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('frenkie de jong', 'fc-barcelona'), 
        ('kante', 'fc-chelsea'), 
        ('kylian mbappe', 'fc-paris-saint-germain'), 
        ('lionel messi', 'fc-barcelona'), 
        ('sadio mane', 'fc-liverpool')
    ],
    '2018/2019': [
        ('david de gea', 'manchester-united'), 
        ('virgil van dijk', 'fc-liverpool'), 
        ('raphael varane', 'real-madrid'), 
        ('sergio ramos', 'real-madrid'), 
        ('marcelo', 'real-madrid'), 
        ('luka modric', 'real-madrid'), 
        ('kante', 'fc-chelsea'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('kylian mbappe', 'fc-paris-saint-germain'), 
        ('cristiano ronaldo', 'juventus-turin'), 
        ('lionel messi', 'fc-barcelona')
    ],
    '2017/2018': [
        ('david de gea', 'manchester-united'), 
        ('dani alves', 'fc-paris-saint-germain'), 
        ('leonardo bonucci', 'ac-mailand'), 
        ('sergio ramos', 'real-madrid'), 
        ('marcelo', 'real-madrid'), 
        ('kevin de bruyne', 'manchester-city'), 
        ('kante', 'fc-chelsea'), 
        ('luka modric', 'real-madrid'), 
        ('lionel messi', 'fc-barcelona'), 
        ('harry kane', 'tottenham-hotspur'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2016/2017': [
        ('manuel neuer', 'fc-bayern-munchen'), 
        ('dani alves', 'juventus-turin'), 
        ('gerard pique', 'fc-barcelona'), 
        ('sergio ramos', 'real-madrid'), 
        ('marcelo', 'real-madrid'), 
        ('luka modric', 'real-madrid'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('toni kroos', 'real-madrid'), 
        ('lionel messi', 'fc-barcelona'), 
        ('luis suarez', 'fc-barcelona'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2015/2016': [
        ('manuel neuer', 'fc-bayern-munchen'), 
        ('dani alves', 'fc-barcelona'), 
        ('thiago silva', 'fc-paris-saint-germain'), 
        ('sergio ramos', 'real-madrid'), 
        ('marcelo', 'real-madrid'), 
        ('paul pogba', 'juventus-turin'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('luka modric', 'real-madrid'), 
        ('lionel messi', 'fc-barcelona'), 
        ('neymar', 'fc-barcelona'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2014/2015': [
        ('manuel neuer', 'fc-bayern-munchen'), 
        ('philipp lahm', 'fc-bayern-munchen'), 
        ('thiago silva', 'fc-paris-saint-germain'), 
        ('david luiz', 'fc-paris-saint-germain'), 
        ('sergio ramos', 'real-madrid'), 
        ('angel di maria', 'manchester-united'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('toni kroos', 'real-madrid'), 
        ('arjen robben', 'fc-bayern-munchen'), 
        ('lionel messi', 'fc-barcelona'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2013/2014': [
        ('manuel neuer', 'fc-bayern-munchen'), 
        ('dani alves', 'fc-barcelona'), 
        ('thiago silva', 'fc-paris-saint-germain'), 
        ('sergio ramos', 'real-madrid'), 
        ('philipp lahm', 'fc-bayern-munchen'), 
        ('xavi', 'fc-barcelona'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('franck ribery', 'fc-bayern-munchen'), 
        ('lionel messi', 'fc-barcelona'), 
        ('zlatan ibrahimovic', 'fc-paris-saint-germain'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2012/2013': [
        ('iker casillas', 'real-madrid'), 
        ('dani alves', 'fc-barcelona'), 
        ('sergio ramos', 'real-madrid'), 
        ('gerard pique', 'fc-barcelona'), 
        ('marcelo', 'real-madrid'), 
        ('xavi', 'fc-barcelona'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('xabi alonso', 'real-madrid'), 
        ('lionel messi', 'fc-barcelona'), 
        ('radamel falcao', 'atletico-madrid'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2011/2012': [
        ('iker casillas', 'real-madrid'), 
        ('dani alves', 'fc-barcelona'), 
        ('nemanja vidic', 'manchester-united'), 
        ('gerard pique', 'fc-barcelona'), 
        ('sergio ramos', 'real-madrid'), 
        ('xavi', 'fc-barcelona'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('xabi alonso', 'real-madrid'), 
        ('lionel messi', 'fc-barcelona'), 
        ('wayne rooney', 'manchester-united'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2010/2011': [
        ('iker casillas', 'real-madrid'), 
        ('maicon', 'inter-mailand'), 
        ('lucio', 'inter-mailand'), 
        ('gerard pique', 'fc-barcelona'), 
        ('sergio ramos', 'real-madrid'), 
        ('wesley sneijder', 'inter-mailand'), 
        ('xavi', 'fc-barcelona'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('lionel messi', 'fc-barcelona'), 
        ('david villa', 'fc-barcelona'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2009/2010': [
        ('iker casillas', 'real-madrid'), 
        ('dani alves', 'fc-barcelona'), 
        ('john terry', 'fc-chelsea'), 
        ('nemanja vidic', 'manchester-united'), 
        ('patrice evra', 'manchester-united'), 
        ('xavi', 'fc-barcelona'), 
        ('andres iniesta', 'fc-barcelona'), 
        ('steven gerrard', 'liverpool'), 
        ('lionel messi', 'fc-barcelona'), 
        ('fernando torres', 'fc-liverpool'), 
        ('cristiano ronaldo', 'real-madrid')
    ],
    '2008/2009': [
        ('iker casillas', 'real-madrid'), 
        ('sergio ramos', 'real-madrid'), 
        ('john terry', 'fc-chelsea'), 
        ('rio ferdinand', 'manchester-united'), 
        ('carles puyol', 'fc-barcelona'), 
        ('xavi', 'fc-barcelona'), 
        ('kaka', 'ac-mailand'), 
        ('steven gerrard', 'liverpool'), 
        ('lionel messi', 'fc-barcelona'), 
        ('fernando torres', 'fc-liverpool'), 
        ('cristiano ronaldo', 'real-madrid')
    ]
}

def normalize_string(s):
    """
    Normalize a string by removing accents and converting to lowercase.
    """
    # Decompose the unicode string into its base characters and accents
    s = unicodedata.normalize('NFKD', s)
    
    # Encode to ASCII to remove accents, then decode back to string
    s = s.encode('ASCII', 'ignore').decode('ASCII')
    
    # Convert to lowercase
    s = s.lower()
    
    return s

# Normalize all player names in the DataFrame and update the 'Player' column
ucl_winners_df['Player'] = ucl_winners_df['Player'].apply(normalize_string)

for season, players in toty_players.items():
    ucl_winners_df.loc[(ucl_winners_df['Season'] == season) & (ucl_winners_df['Player'].isin(players)), 'TOTY'] = 1

# Function to update the TOTY column
def update_toty_column(df, toty_dict):
    for season, players in toty_dict.items():
        for player, team in players:
            normalized_player = normalize_string(player)
            df.loc[(df['Season'] == season) & 
                   (df['Player'] == normalized_player) & 
                   (df['Team'] == team), 'TOTY'] = 1
    return df

# Update the TOTY column based on the dictionary
ucl_winners_df = update_toty_column(ucl_winners_df, toty_players)

ucl_winners_df.head(20)

C:\Users\haiaj\AppData\Local\Temp\ipykernel_29624\2955811618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucl_winners_df['TOTY'] = 0
C:\Users\haiaj\AppData\Local\Temp\ipykernel_29624\2955811618.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ucl_winners_df['Player'] = ucl_winners_df['Player'].apply(normalize_string)


,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup,TOTY
1,victor valdes,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
6,martin caceres,21,Centre-Back,2,Uruguay,51,23,-,-,3,...,-,2.13,"1,523'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
8,gerard pique,21,Centre-Back,3,Spain,50,45,3,1,7,...,1,2.27,"3,932'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
9,rafa marquez,29,Centre-Back,4,Mexico,41,37,3,3,7,...,8,2.49,"3,020'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
10,carles puyol,30,Centre-Back,5,Spain,49,45,1,3,8,...,6,2.29,"3,768'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,1
11,sylvinho,34,Left-Back,16,Brazil,49,30,1,2,-,...,4,2.20,"1,962'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
13,dani alves,25,Right-Back,20,Brazil,54,54,5,14,13,...,4,2.35,"4,702'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
14,eric abidal,28,Left-Back,22,France,39,32,-,1,6,...,2,2.09,"2,652'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
15,sergio busquets,19,Defensive Midfield,28,Spain,53,41,3,1,12,...,8,2.41,"2,909'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
16,andres iniesta,24,Central Midfield,8,Spain,44,43,5,17,5,...,19,2.40,"3,287'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0


In [161]:
ucl_winners_df.head()

,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup,TOTY
1,victor valdes,26,Goalkeeper,1,Spain,52,49,-,-,3,...,-,2.35,"4,410'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
6,martin caceres,21,Centre-Back,2,Uruguay,51,23,-,-,3,...,-,2.13,"1,523'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
8,gerard pique,21,Centre-Back,3,Spain,50,45,3,1,7,...,1,2.27,"3,932'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
9,rafa marquez,29,Centre-Back,4,Mexico,41,37,3,3,7,...,8,2.49,"3,020'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0
10,carles puyol,30,Centre-Back,5,Spain,49,45,1,3,8,...,6,2.29,"3,768'",FC Barcelona,2008/2009,fc-barcelona,laliga,1,1,0


In [222]:
# Define the function to search for players by name and season
def search_player_season(df, player_name, season_year):
    """
    Search for player records based on name and season.

    Parameters:
    df (pd.DataFrame): The DataFrame to search within.
    player_name (str): The name of the player.
    season_year (str): The season year (e.g., '2022/2023').

    Returns:
    pd.DataFrame: Filtered DataFrame containing records for the specified player and season.
    """
    # Filter the DataFrame
    records = df[(df['Player'].str.contains(player_name, case=False)) & (df['Season'] == season_year)]
    
    # Display the records
    print(records)
    
    return records

# Example usage
player_name = 'kylian mbappe'
season_year = '2021/2022'
search_player_season(ucl_winners_df, player_name, season_year)

              Player Age        Position Kit Number Nationality In Squad  \
55851  kylian mbappe  22  Centre-Forward          7      France       46   

       Appearances Goals Assists Yellow Cards  ... Substituted Off   PPG  \
55851           46    39      26           11  ...              13  2.22   

      Minutes Played                 Club     Season                    Team  \
55851         3,912'  Paris Saint-Germain  2021/2022  fc-paris-saint-germain   

        League Champions League Domestic Cup TOTY  
55851  ligue-1                0            0    1  

[1 rows x 23 columns]


,Player,Age,Position,Kit Number,Nationality,In Squad,Appearances,Goals,Assists,Yellow Cards,...,Substituted Off,PPG,Minutes Played,Club,Season,Team,League,Champions League,Domestic Cup,TOTY
55851,kylian mbappe,22,Centre-Forward,7,France,46,46,39,26,11,...,13,2.22,"3,912'",Paris Saint-Germain,2021/2022,fc-paris-saint-germain,ligue-1,0,0,1


In [173]:
len(ucl_winners_df_

24895